In [1]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

# Benchmark Over Network

## Create Benchmark Server

create server control client + server (needed for turning files into urls)

In [3]:
# dont_test

server_control_host = Host(name="192.168.178.113", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(
    servers.FastAPIUvicornServer(host="192.168.178.113", port=5001)
)

<IPython.core.display.Javascript object>

## Create Files and Urls

In [4]:
# dont_test

byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)
epoch.files = server_control_client.get_or_create_files(epoch)
epoch.urls = [server.file_to_url(file) for file in epoch.files]

<IPython.core.display.Javascript object>

## Modify Files

In [5]:
# dont_test

for epoch_file in epoch.files:
    epoch_file.port = server.port
    epoch_file.hostname = server.host

<IPython.core.display.Javascript object>

## Create Control Client

In [6]:
# dont_test

client_control_host = Host(name="localhost", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_client = clients.HttpxClient(name="httpx keepalive 10")
# benchmark_client = clients.AioHttpClient(name="aiohttp 400 ethernet")
# benchmark_client = clients.WrkClient(
#     name="wrk ethernet", host=server.host, port=server.port
# )

<IPython.core.display.Javascript object>

## Create Result

In [7]:
# dont_test

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

In [8]:
# epoch.urls[:5]

<IPython.core.display.Javascript object>

In [9]:
# result.elapsed = 2.80
# result.readable_bytes_per_second

<IPython.core.display.Javascript object>

## Measure Server with Client

In [10]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)

<IPython.core.display.Javascript object>

## Stop Server

In [9]:
# dont_test

server_control_client.stop_server(server)

True

<IPython.core.display.Javascript object>

## Save Result

We are using a sqlite database for saving benchmark results.

In [11]:
# dont_test

repository = SqliteRepository.build_repository("results.db")
register_default_tables(repository)
results_table = repository.tables["result"]

<IPython.core.display.Javascript object>

In [12]:
# dont_test

result_id = results_table.add_result(result)

<IPython.core.display.Javascript object>

In [13]:
# dont_test

results_from_database = results_table.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)

,server,client,elapsed,file_size_h,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10.352465,9.54MB,34.55MB
1,fastAPI/uvicorn,httpx,8.318073,9.54MB,42.99MB
2,fastAPI/uvicorn,aiohttp,9.394115,9.54MB,38.07MB
3,fastAPI/uvicorn,aiohttp 400,8.790079,9.54MB,40.69MB
4,fastAPI/uvicorn,wrk,6.617528,9.54MB,54.04MB
5,fastAPI/uvicorn,wrk ethernet,2.795011,9.54MB,127.95MB
6,fastAPI/uvicorn,httpx,5.734141,9.54MB,62.37MB
7,fastAPI/uvicorn,aiohttp 400 ethernet,5.588128,9.54MB,64.0MB
8,fastAPI/uvicorn,httpx keepalive 10,5.657740,9.54MB,63.21MB


<IPython.core.display.Javascript object>